In [109]:
import os

while not os.getcwd().endswith("hackathon"):
    os.chdir("..")
    print(f"Now in {os.getcwd()}")

In [110]:
from hackathon.utils.settings.settings_provider import SettingsProvider
import warnings
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

with warnings.catch_warnings(action="ignore"):
    from langchain_ibm import ChatWatsonx
import dotenv
from langchain.schema import Document
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


dotenv.load_dotenv();

In [111]:
settings_provider = SettingsProvider()

In [4]:
llm = ChatWatsonx(
    model_id=settings_provider.get_ibm_model_name(),  # type: ignore
    url=settings_provider.get_ibm_endpoint_url(),  # type: ignore
    project_id=settings_provider.get_ibm_project_id(),  # type: ignore
)

In [112]:
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}

embeddings = HuggingFaceEmbeddings(
    model_name=settings_provider.get_embeddings_model_name(),
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [20]:
# import faiss
# from langchain_community.docstore.in_memory import InMemoryDocstore
# from langchain_community.vectorstores import FAISS

# index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

# vectorstore = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

In [115]:
vector_store = FAISS.load_local(
    "data/debug_vectorstore", embeddings, allow_dangerous_deserialization=True
)

In [47]:
# # from uuid import uuid4

# from langchain_core.documents import Document

# document_1 = Document(
#     page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
#     metadata={"source": "tweet", "options": ["pancakes", "eggs"]},
# )

# document_2 = Document(
#     page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
#     metadata={"source": "news", "options": ["weather", "forecast"]},
# )

# document_3 = Document(
#     page_content="Building an exciting new project with LangChain - come check it out!",
#     metadata={"source": "tweet", "options": ["LangChain", "project"]},
# )

# documents = [
#     document_1,
#     document_2,
#     document_3,
# ]
# uuids = [str(uuid4()) for _ in range(len(documents))]

# vector_store = FAISS.from_documents(documents=documents, embedding=embeddings)
# vector_store.save_local(folder_path="data/debug_vectorstore")

In [128]:
vector_store.similarity_search(
    "dcsvfssadsvfsssdaer",
    k=3,
    filter=lambda x: x.get("chef_name", "") == "Alessandra Quanti",
)

[Document(id='d3a548ca-8f7d-44b3-9415-5801267b4c29', metadata={'header_1': 'Menu', 'header_2': 'Sinfonia Quantistica delle Stelle', 'source': 'Sapore del Dune.pdf', 'chef_name': 'Alessandra Quanti', 'restaurant_name': 'sapore del dune', 'planet_name': 'Tatooine', 'licences': ['Psionica II', 'Temporale I', 'Gravitazionale I', 'Quantistica 13', 'Luce II']}, page_content='### Ingredienti  \nShard di Prisma Stellare\nLattuga Namecciana\nRadici di Singolarità  \n-----  \nFibra di Sintetex\nCarne di Balena spaziale\nTeste di Idra\nNettare di Sirena\nSale Temporale  \n### Tecniche  \nMarinatura Temporale Sincronizzata\nCottura Sottovuoto Frugale Energeticamente Negativa\nCottura a Vapore Termocinetica Multipla\nCottura a Vapore Ecodinamico Bilanciato'),
 Document(id='ab6e7fcf-2278-403f-b45c-cc4a0f701a8e', metadata={'header_1': 'Menu', 'header_2': "Sinfonia Quantistica dell'Universo", 'source': 'Sapore del Dune.pdf', 'chef_name': 'Alessandra Quanti', 'restaurant_name': 'sapore del dune', 'plan

In [171]:
# Alternative: If you want exact array match

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5, "fetch_k": vector_store.index.ntotal})

retriever.invoke(
    "Radici di Singolaavfavsavfdfdafrità",
    filter=lambda x: "Alessandra Quanti" in x.get("chef_name", ""),
)

[Document(id='d3a548ca-8f7d-44b3-9415-5801267b4c29', metadata={'header_1': 'Menu', 'header_2': 'Sinfonia Quantistica delle Stelle', 'source': 'Sapore del Dune.pdf', 'chef_name': 'Alessandra Quanti', 'restaurant_name': 'sapore del dune', 'planet_name': 'Tatooine', 'licences': ['Psionica II', 'Temporale I', 'Gravitazionale I', 'Quantistica 13', 'Luce II']}, page_content='### Ingredienti  \nShard di Prisma Stellare\nLattuga Namecciana\nRadici di Singolarità  \n-----  \nFibra di Sintetex\nCarne di Balena spaziale\nTeste di Idra\nNettare di Sirena\nSale Temporale  \n### Tecniche  \nMarinatura Temporale Sincronizzata\nCottura Sottovuoto Frugale Energeticamente Negativa\nCottura a Vapore Termocinetica Multipla\nCottura a Vapore Ecodinamico Bilanciato'),
 Document(id='2ba8bbbf-ae68-41f2-ac1f-a4dd518019ad', metadata={'header_1': 'Menu', 'header_2': 'Evanescenza Quantica', 'source': 'Sapore del Dune.pdf', 'chef_name': 'Alessandra Quanti', 'restaurant_name': 'sapore del dune', 'planet_name': 'Tat

In [46]:
query_ex = "Quali piatti del ristorante Cosmica Essenza sono preparati utilizzando i Funghi dell’Etere o la tecnica di Marinatura Temporale Sincronizzata?"

In [117]:
import numpy as np


embedding_dim = vector_store.index.d
dummy_vector = np.zeros(embedding_dim)

# Retrieve all documents by setting k to the total number of documents
docs_and_scores = vector_store.similarity_search_with_score_by_vector(
    dummy_vector, k=vector_store.index.ntotal
)

docs_and_scores[:10]

[(Document(id='cbf6fc24-db1e-4521-8e18-35d4185403da', metadata={'header_1': 'Capitolo 3: Tecniche di Preparazione', 'header_2': 'Tecniche di Impasto', 'header_3': 'Impasto Gravitazionale Vorticoso', 'source': 'Manuale di cucina.pdf', 'source_of_truth': True}, page_content="Come funziona: utilizza campi gravitazionali artificialmente generati per creare vortici che mescolano gli  \ningredienti in modo uniforme, sfruttando la rotazione a spirale per incorporare facilmente aria e umidità.  \nVantaggi: raggiunge una consistenza perfetta e omogenea senza il rischio di sovraccarico meccanico;  \nideale per impasti leggeri come soufflé e torte.  \nSvantaggi: richiede un’accurata regolazione delle forze gravitazionali per evitare eruzioni improvvise di  \nmateriale nell'ambiente di lavoro.  \n-----  \nCome funziona: manipola direttamente le catene molecolari degli ingredienti attraverso catalizzatori  \nsintetici, promuovendo la coesione spontanea e naturale dell'impasto.  \nVantaggi: garantis

[]

In [36]:
import chromadb
chroma_client = chromadb.Client()

In [37]:
collection = chroma_client.create_collection(name="my_collection")

In [46]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id3", "id5"],
    metadatas=[{"fruit": ["pineapple"]}, {"fruit": "orange"}],
)

ValueError: Expected metadata value to be a str, int, float or bool, got ['pineapple'] which is a list in add.

In [34]:


# Example documents
documents = [
    "Document about regression and classification",
    "Document about CNN architecture",
    "Document about both regression and CNN"
]

# Method 1: Using lists in metadata (Recommended)
metadata_list = [
    {
        "techniques": ["regression", "classification"],
        "difficulty": "intermediate"
    },
    {
        "techniques": ["CNN"],
        "difficulty": "advanced"
    },
    {
        "techniques": ["regression", "CNN"],
        "difficulty": "advanced"
    }
]

# Add documents with metadata using lists
collection.add(
    documents=documents,
    ids=["doc1", "doc2", "doc3"],
    metadatas=metadata_list
)

# Query examples
# 1. Find documents about regression
results = collection.query(
    query_texts=["regression"],
    where={"techniques": {"$in": ["regression"]}}
)

# 2. Find advanced documents about CNN
results_advanced_cnn = collection.query(
    query_texts=["CNN architecture"],
    where={
        "$and": [
            {"techniques": {"$in": ["CNN"]}},
            {"difficulty": "advanced"}
        ]
    }
)